In [2]:
def csv_FileReader(csvFile):
    import csv
    with open(csvFile, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            print(row)
csvFile = 'accounts.csv'
csv_FileReader(csvFile)

def csv_FileWriter(csvFile): 
    
    import csv 
    with open(csvFile, 'w') as file:
        writer = csv.writer(file)


['amohr', '1337', '43.00', '0666', '101.45', '1729', '5000.00', '5000']
['bbaggins', '2890', '15345.49', '2941', '########', '3021', '0.00', '50000']
['emusk', '0001', '21588738', '0002', '########', '0003', '435678.6', '10000000']


In [1]:
class Account:
    '''
    Represents a generic financial account

    attributes: accountID a string of 4 integers, balance as a float or int, interest as a float or int
    '''
    def __init__(self, accountID = '', balance = 0, interest = 0):

        if not isinstance(accountID, str):
            raise TypeError('Account id must be a string')
        
        if len(accountID) != 4:
            raise ValueError('Account id must contain 4 digits')

        for number in accountID:
            if number not in '0123456789':
                raise ValueError('Account id must contain only numbers')
        
        if not isinstance(balance, (int, float)):
            raise TypeError('balance must be an integer or float')
        
        if not isinstance(interest, (int, float)):
            raise TypeError('interest must be an integer or float')
        
        if interest < 0:
            raise ValueError('Interest cannot be negative')
        
        self.accountID = accountID
        self.balance = balance
        self.interest = interest
    
    def __str__(self):
        return f'Account ID: {self.accountID} Balance: {self.balance} Interest: {self.interest}%'
            

class Checking(Account):
    '''
    Represents a checking account inheriting the Account class

    attributes: inherited attributes from Account class
    '''
    def __init__(self, accountID = '', balance = 0):
        super().__init__()
        self.interest = 0

class Savings(Account):
    '''
    Represents a savings account inheriting the Account class

    attributes: inherited attributes from Account class
    '''
    def __init__(self, accountID = '', balance = 0):
        super().__init__()
        self.interest = 1

class Credit(Account):
    '''
    Represents a credit account inheriting the Account class

    attributes: inherited attributes from Account class and a credit_limit which is an int or float
    '''
    def __init__(self, accountID = '', balance = 0, interest = 0, credit_limit = 0):
        super().__init__(accountID, balance, interest)
        
        if not isinstance(credit_limit, (int, float)):
            raise TypeError('Credit limit must be an int or float')
        
        if credit_limit < balance:
            raise ValueError("Credit limit must be greater than or equal to the balance")

        self.credit_limit = credit_limit

    def __str__(self):
        s = super().__str__()
        return f'{s} Credit limit: {self.credit_limit}'

class Customer:
    '''
    Respresents a customer account

    attributes: username as a string, checking as a Checking object, savings as a Savings object, credit as a Credit object
    '''        
    def __init__(self, username = '', checking = Checking() , savings = Savings(), credit = Credit()):
        
        if not isinstance(username, str):
            raise TypeError('username must be a string')
        
        if not isinstance(checking, Checking):
            raise TypeError('checking must be a Checking object')

        if not isinstance(savings, Savings):
            raise TypeError('savings must be a Savings object')

        if not isinstance(credit, Credit):
            raise TypeError('credit must be a Credit object')

        self.username = username
        self.checking = checking
        self.savings = savings
        self.credit = credit
    

In [22]:
def interface():
    choice = ''
    while choice != '6':
        print('Select an option:', flush=True)
        print("1: Deposit ", flush=True)
        print("2: Withdrawl", flush=True)
        print("3: Credit Card Charge", flush=True)
        print("4: Credit Card Payment", flush=True)
        print("5: View Customers", flush=True)
        print("6: Exit/Shut Down", flush=True)
        print(flush=True)

        choice = input()

        if choice == '1':
            deposit()
            # Try: Deposit()
            # Except: print("An error occured")
        elif choice == '2':
            withdrawl()
        elif choice == '3':
            creditcardcharge()
        elif choice == '4':
            creditcardpayment()
        elif choice == '5':
            viewcustomers()
        elif choice == '6':
            print("Shutting Down...", flush=True)
        else:
            print("Select and actual option.")

interface()

Select an option:
1: Deposit 
2: Withdrawl
3: Credit Card Charge
4: Credit Card Payment
5: View Customers
6: Exit/Shut Down



NameError: name 'viewcustomers' is not defined

In [ ]:
def importCSV():

In [ ]:
def viewCustomers():

In [ ]:
def deposit():

In [ ]:
def withdrawl():

In [ ]:
def creditCardCharge():

In [ ]:
def creditCardPayment():

In [30]:
def csv_FileWriter(csvFile): 
    
    import csv 
    with open(csvFile, 'w') as file:
        writer = csv.writer(file)
        
def interfaceExit():
    def openfile():
        print(f'Enter file address:', flush=True)
        file_add = input()
        
        try:
            read_file = open(file_add)
            print(f'File successfully opened.')
        
        except: 
            print(f'Invalid file address. Please try again.')
            openfile()

        csv_FileWriter(file_add)

interfaceExit()